In [2]:
import pandas as pd
import numpy as np 
import sys
sys.path.append('../../Function')
import function
from sklearn.metrics import mean_absolute_error
from keras.layers import Dense, Dropout
from keras.models import Sequential


In [3]:
df = pd.read_excel('../../../site_info_ver_5_light.xlsx')

In [15]:
dataset = df.copy()
sites = dataset.Site_Name.unique()
site_name = 'Gan HaShlosha'
target = 'Israelis_Count'
target_title = 'Israelis'
dataset = dataset.loc[dataset.Site_Name==site_name]


dataset = dataset.drop(['Total','Tourists_Count'],axis=1)
dataset = dataset.drop(dataset.filter(regex='nox').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='so2').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='pm2.5').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='Ashkelon').columns, axis=1)

print('model on site',site_name)
print('Focus on ',target_title)


dataset = function.remove_unique_one(dataset)
dataset = function.remove_high_corr(dataset, target,0.3)
dataset = function.split_date(dataset)
dataset = function.last_year_entries_info(dataset,target)
dataset.drop("Date",axis=1,inplace=True)
print('shape of dataset',dataset.shape)
dataset.dropna(inplace=True)
dataset = dataset.astype({'day': 'category'})
dataset = dataset.astype({'month': 'category'})
dataset = dataset.astype({'year': 'category'})
dataset = dataset.drop('day', axis=1)
dataset = dataset.drop('month', axis=1)
dataset = dataset.drop('year', axis=1)
print(dataset.dtypes)
print(dataset.shape)
dataset = pd.get_dummies(dataset)

print('shape of dataset',dataset.shape)
print('features :',dataset.columns)
dataset[['Israelis_Count']].describe()

model on site Gan HaShlosha
Focus on  Israelis
Add Last year visitors Successfully
shape of dataset (971, 14)
is_weekend                       int64
operations                       int64
is_jewish_holiday                int64
is_muslims_holiday               int64
Haifa_pm10                     float64
Tel_Aviv-Yafo_pm10_exceeded      int64
Haifa_pm10_exceeded              int64
Season_spring                    int64
Season_summer                    int64
Last_year_visitors             float64
Israelis_Count                   int64
dtype: object
(971, 11)
shape of dataset (971, 11)
features : Index(['is_weekend', 'operations', 'is_jewish_holiday', 'is_muslims_holiday',
       'Haifa_pm10', 'Tel_Aviv-Yafo_pm10_exceeded', 'Haifa_pm10_exceeded',
       'Season_spring', 'Season_summer', 'Last_year_visitors',
       'Israelis_Count'],
      dtype='object')


,Israelis_Count
count,971.000000
mean,751.360453
std,854.822104
min,2.000000
25%,92.000000
50%,398.000000
75%,1154.000000
max,3992.000000


In [16]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = dataset.drop(target,axis=1)
y = dataset[target]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=312148513)

X_train_scaler = MinMaxScaler()
X_test_scaler = MinMaxScaler()

X_train_scaled = X_train_scaler.fit_transform(X_train)
X_test_scaled = X_test_scaler.fit_transform(X_test)

y_train_scaled = np.log(y_train+0.001)
y_test_scaled = np.log(y_test+0.001)

dataset_train = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
dataset_test = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)



STDs

In [5]:
print("train STD " + str(np.std(y_train)))
print("test STD " + str(np.std(y_test)))

train STD 878.868933052146
test STD 877.345127851053


MLR Test

In [17]:
from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(X_train_scaled,y_train)

prediction =mlr.predict(X_test_scaled)

res = pd.DataFrame(
    data={
        'Prediction':prediction,
        'Actual': y_test.values
    },
    index=y_test.index
)

print('rmse',function.get_rmse(res.Prediction, res.Actual))
print('std',np.std(res.Actual))

coef = sorted( list(zip(np.round(mlr.coef_,3).T,X_train.columns)))
coef

rmse 505.14395090098446
std 809.4061334800889


[(-123.341, 'Haifa_pm10'),
 (-114.362, 'operations'),
 (-3.681, 'Haifa_pm10_exceeded'),
 (121.738, 'Tel_Aviv-Yafo_pm10_exceeded'),
 (176.055, 'is_muslims_holiday'),
 (181.432, 'Season_spring'),
 (419.133, 'is_weekend'),
 (604.467, 'is_jewish_holiday'),
 (726.644, 'Season_summer'),
 (1645.948, 'Last_year_visitors')]

MLR Train

In [18]:
prediction =mlr.predict(X_train_scaled)

res = pd.DataFrame(
    data={
        'Prediction':prediction,
        'Actual': y_train.values
    },
    index=y_train.index
)

print('rmse',function.get_rmse(res.Prediction, res.Actual))
print('std',np.std(res.Actual))

coef = sorted( list(zip(np.round(mlr.coef_,3).T,X_train.columns)))
coef

rmse 530.8218461138699
std 865.270132513319


[(-123.341, 'Haifa_pm10'),
 (-114.362, 'operations'),
 (-3.681, 'Haifa_pm10_exceeded'),
 (121.738, 'Tel_Aviv-Yafo_pm10_exceeded'),
 (176.055, 'is_muslims_holiday'),
 (181.432, 'Season_spring'),
 (419.133, 'is_weekend'),
 (604.467, 'is_jewish_holiday'),
 (726.644, 'Season_summer'),
 (1645.948, 'Last_year_visitors')]